In [ ]:
import os
import tensorflow as tf

file_pattern = '../mnt/tess/astronet/tfrecords-38-{0}'
tces_file = '../mnt/tess/astronet/tces-v14-all.csv'

In [ ]:
import pandas as pd

tce_table = pd.read_csv(tces_file, header=0, low_memory=False).set_index('Astro ID')

In [ ]:
for split in ['train', 'val']:
  filenames = tf.io.gfile.glob(file_pattern.format(split) + '/*')
  
  writers = {}
  outfiles = {}
  for i in range(1, 4):
    dr = file_pattern.format('y' + str(i) + '-' + split)
    tf.io.gfile.makedirs(dr)
    outf = dr + '/00000-of-00001'
    wr = tf.io.TFRecordWriter(outf)
    wr.__enter__()
    outfiles[i] = outf
    writers[i] = wr

  try:
    counts = {i: 0 for i in range(1, 4)}
    for filename in filenames:
      print(filename, end='')
      tfr = tf.data.TFRecordDataset(filename)
      for r in tfr:
        stat = ' '.join([f'{k}:{v}' for k, v in counts.items()])
        print(f'\r{filename}: {stat}', end='')
#         print('\r' + filename + ' '.join([(str(k) + ':' + str(v)) for k, v in counts.items()]), end='')
        ex = tf.train.Example.FromString(r.numpy())
        for k, f in ex.features.feature.items():
          if k == "astro_id":
            ex_id = f.int64_list.value[0]
            yr = tce_table[tce_table.index == ex_id].Year.values[0]
            counts[yr] += 1
            writers[yr].write(ex.SerializeToString())
            break
      print('')
  finally:
    for _, wr in writers.items():
      wr.__exit__()